In [7]:
import os
from dendro.client import submit_job, DendroJobDefinition, DendroJobRequiredResources, DendroJobInputFile, DendroJobOutputFile, DendroJobParameter

In [8]:
dendro_api_key = os.environ.get('DENDRO_API_KEY')
if not dendro_api_key:
    # prompt the user for the API key
    dendro_api_key = input('Please enter your Dendro API key: ')
    os.environ['DENDRO_API_KEY'] = dendro_api_key

In [9]:
def process_session(url: str):
    input_url = url

    service_name = 'hello_world_service'
    app_name = 'hello_neurosift'
    processor_name = 'tuning_analysis_000363'
    job_definition = DendroJobDefinition(
        appName=app_name,
        processorName=processor_name,
        inputFiles=[
            DendroJobInputFile(
                name='input',
                url=input_url,
                fileBaseName='input.nwb'
            )
        ],
        outputFiles=[
            DendroJobOutputFile(
                name='output',
                fileBaseName='output.nwb.lindi.tar'
            )
        ],
        parameters=[
            DendroJobParameter(
                name='units_path',
                value='/units'
            ),
            DendroJobParameter(
                name='behavior_paths',
                value=[
                    '/acquisition/BehavioralTimeSeries/Camera0_side_JawTracking',
                    '/acquisition/BehavioralTimeSeries/Camera0_side_NoseTracking',
                    '/acquisition/BehavioralTimeSeries/Camera0_side_TongueTracking'
                ]
            ),
            DendroJobParameter(
                name='behavior_dimensions',
                value=[
                    1,
                    1,
                    1
                ]
            ),
            DendroJobParameter(
                name='behavior_output_prefixes',
                value=[
                    'jaw',
                    'nose',
                    'tongue'
                ]
            )
        ]
    )
    required_resources = DendroJobRequiredResources(
        numCpus=4,
        numGpus=0,
        memoryGb=4,
        timeSec=60 * 50
    )

    job = submit_job(
        service_name=service_name,
        job_definition=job_definition,
        required_resources=required_resources,
        target_compute_client_ids=None,
        tags=[],
        skip_cache=False,
        rerun_failing=True,
        delete_failing=True
    )

    return job

In [ ]:
# https://neurosift.app/?p=/nwb&url=https://api.dandiarchive.org/api/assets/0eab806c-c5c3-4d01-bd7c-15e328a7e923/download/&dandisetId=000363&dandisetVersion=draft
input_url = 'https://api.dandiarchive.org/api/assets/0eab806c-c5c3-4d01-bd7c-15e328a7e923/download/'

job = process_session(input_url)
print(f'{job.job_url} {job.status}')

In [ ]:
import dandi.dandiarchive as da

# Example of listing assets in a DANDI dataset
dandiset_id = "000363"

parsed_url = da.parse_dandi_url(f"https://dandiarchive.org/dandiset/{dandiset_id}")
max_num = 10  # limit the number of assets to be listed to not overuse the DANDI API
with parsed_url.navigate() as (client, dandiset, assets):
    if dandiset is None:
        raise ValueError(f"Failed to find dandiset {dandiset_id}")
    asset_index = 0
    for asset in assets:
        path = asset.path
        if path.endswith('.nwb'):
            url = asset.get_content_url()
            ns_url = f'https://neurosift.app/?p=/nwb&url={url}&dandisetId={dandiset_id}'
            print(f'{asset_index}: {path} {url}')
            print(f'{ns_url}')
            job = process_session(url)
            print(f'{job.job_url} {job.status}')
            print('')
        asset_index += 1
        if asset_index >= max_num:
            print(f"Reached the limit of {max_num} assets")
            break
